In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import os
import numpy as np
import glob
from PIL import Image, ExifTags

# pip install torchsummary
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.transforms.functional as fn
from torchvision import models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from time import time
import pandas as pd
import random

import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

from matplotlib.patches import Polygon, Rectangle
import pylab
pylab.rcParams['figure.figsize'] = (6,6)

# Own imports 
from config import * 
from utils import *
from data_loader import TacoDataset
from eval import *

# speed-up using multithreads
cv2.setUseOptimized(True);
cv2.setNumThreads(8);

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
img_size = 256

train_transform = A.Compose([
    A.Resize(img_size, img_size),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=30, p=0.7),
    A.RandomBrightnessContrast(p=0.2),
    A.RGBShift(10, 10, 10, p=0.3),
    A.GaussNoise(p=0.5),
    A.Normalize(), # If you want to visualize - comment this line 
    ToTensorV2(),
], bbox_params=A.BboxParams(format='pascal_voc',
                            label_fields=['labels'],
                            min_visibility=0.3, # min visibility of the original area in case of a crop
                           )
)

test_transform = A.Compose([
    A.Resize(img_size, img_size),
    A.Normalize(),
    ToTensorV2(),
], bbox_params=A.BboxParams(format='pascal_voc',
                            label_fields=['labels'],
                           )
)

In [3]:
trainset = TacoDataset( 'train', transforms=train_transform, test_size=0.2) # test_transform for no augment
valset   = TacoDataset('val', transforms=test_transform, test_size=0.2)
testset  = TacoDataset('test', transforms=test_transform, test_size=0.2)

train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8,collate_fn=lambda x: x)# persistent_workers=True, pin_memory=True)
val_loader = DataLoader(valset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8,collate_fn=lambda x: x)# persistent_workers=True, pin_memory=True)
test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8,collate_fn=lambda x: x)# persistent_workers=True, pin_memory=True)

In [4]:
loss = nn.BCELoss()

model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
model = transfer_model_set(model, freeze_convs=True)
optimizer = torch.optim.Adam(model.parameters(), HEAD_LEARNING_RATE)

Freezing Convs


In [5]:
out_dict = train(model,
                          train_loader,
                          test_loader,
                          loss,
                          optimizer,
                          NUM_EPOCHS, 
                          save_model=True, 
                          model_name='densenet121_head_Adam')

  0%|          | 0/5 [00:00<?, ?epoch/s]

  0%|          | 0/57 [00:00<?, ?it/s]

Total Number of Region Proposals: 796


/zhome/dc/f/181253/course02514/venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Total Number of Region Proposals: 609
Total Number of Region Proposals: 513
Total Number of Region Proposals: 840
Total Number of Region Proposals: 967
Total Number of Region Proposals: 609
Total Number of Region Proposals: 737
Total Number of Region Proposals: 710
Total Number of Region Proposals: 66
Total Number of Region Proposals: 257
Total Number of Region Proposals: 629
Total Number of Region Proposals: 691
Total Number of Region Proposals: 812
Total Number of Region Proposals: 224
Total Number of Region Proposals: 235
Total Number of Region Proposals: 898
15
16


ValueError: Expected argument `preds` and `target` to have the same length